#### Modifications for this horrible intcode problem: 

Add relative mode (mode = 2):
- Instead, they count from a value called the relative base. The relative base starts at 0.

The relative base is modified with the relative base offset instruction:

- Opcode 9 adjusts the relative base by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.

In [129]:
def mode_check_1(mode, list_vals, init_val, rel_base):
    if mode == 0:
        value =  list_vals[init_val + 1]

    if mode == 1:
        value =  init_val + 1

    if mode == 2:
        value =  rel_base + list_vals[init_val + 1]
    
    return value

In [130]:
test_a = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
test_a[2 + 1]

-1

In [131]:
# exmaple two should output 1985
mode_check_1(2, [204,-34], 0, 2019)

1985

In [132]:
def mode_check_2(mode, list_vals, init_val, rel_base):
    if mode == 0:
        value = list_vals[init_val + 2]

    if mode == 1:
        value = init_val + 2

    if mode == 2:
        value = rel_base + list_vals[init_val + 2]
        
    return value

In [133]:
# This should handle 203 error
# Will allow for the handling of mode 2
def val_literal(mode, list_vals, init_val, rel_base):
    if mode == 0:
        value = list_vals[init_val + 3]

    if mode == 1:
        raise Exception("not allowed")

    if mode == 2:
        value = rel_base + list_vals[init_val + 3]
        
    return value    

In [134]:
def intcode_pc(list_vals, init_val, rel_base):
    
    """Built in Day 7"""
    
    while True:

        # identify the opt_code_str
        opt_code_str = str(list_vals[init_val])

        # take actual opt code: 
        opt_code = int(opt_code_str[-2:])

        # modes can be 0 == position or 1 == immediate 
        # handle mode with try / except
        mode_3, mode_2, mode_1 = 0, 0, 0

        # overwrite 0s UNLESS type error, then we know we had trailing 0s
        # saw example here: https://p.szy.io/MrmgEA/python
        try:
            mode_1 = int(opt_code_str[-3])
            mode_2 = int(opt_code_str[-4])
            mode_3 = int(opt_code_str[-5])

        except IndexError:
            pass


        # handling each type of opt code
        if opt_code == 1:

            # the third param provides the writing point: we always look at the 3rd value after opt code to write
            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            # get sum vals
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)
            ans = list_vals[val1] + list_vals[val2]

            # overwrite
            list_vals[write_ix] = ans

            init_val += 4

        elif opt_code == 2:

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)
            ans = list_vals[val1] * list_vals[val2]

            # overwrite
            list_vals[write_ix] = ans
      
            init_val += 4

        elif opt_code == 3:

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)
            list_vals[write_ix] = int(input())
            init_val += 2


        elif opt_code == 4:
            # output val that is at 'init_val + 1' index: think i need t be able to handle a non-0 val....
            # possibility of mode == 1?

            # something off with mode not being picked up properly
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            print(f"Val in 4 is {list_vals[val1]}")
   
            init_val += 2        

        elif opt_code == 5:

            # jump-if-true
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            if list_vals[val1] != 0:
                init_val = list_vals[val2]
            else:
                init_val += 3

        elif opt_code == 6:

            # jump-if-false
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            if list_vals[val1] == 0:
                init_val = list_vals[val2]
            else:
                init_val += 3

        elif opt_code == 7:

            # less than
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            ans = 1 if list_vals[val1] < list_vals[val2] else 0
            list_vals[write_ix] = ans
            init_val += 4

        elif opt_code == 8:

            # equals
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            ans = 1 if list_vals[val1] == list_vals[val2] else 0
            list_vals[write_ix] = ans
            init_val += 4
            
        elif opt_code == 9:

            # update relative base
            rel_base += list_vals[init_val +1]
            init_val += 2
            
        elif opt_code == 99:
            break

        else:

            print("Something is broken!")

In [135]:
intcode_pc([104,1125899906842624,99], init_val = 0, rel_base = 0)

Val in 4 is 1125899906842624


In [136]:
intcode_pc([1102,34915192,34915192,7,4,7,99,0], init_val = 0, rel_base = 0)

Val in 4 is 1219070632396864


In [137]:
test_case1 = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
extra_0_list = [[0] * 9999999]

In [138]:
test_again = test_case1 + extra_0_list[0]

In [139]:
intcode_pc(list_vals = test_again, init_val = 0, rel_base=0)

Val in 4 is 109
Val in 4 is 1
Val in 4 is 204
Val in 4 is -1
Val in 4 is 1001
Val in 4 is 100
Val in 4 is 1
Val in 4 is 100
Val in 4 is 1008
Val in 4 is 100
Val in 4 is 16
Val in 4 is 101
Val in 4 is 1006
Val in 4 is 101
Val in 4 is 0
Val in 4 is 99
